In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO


import numpy as np

from ensemble_boxes import *

apex is not installed


In [31]:
dir = './ensemble'
# ensemble csv files
submission_files = [#'/submission_12_1.csv', # cascade_mask_rcnn_(backbone-swin-t)_(epoch-40) lb: 0.4063
                    '/submission_40.csv',  # cascade_mask_rcnn_(backbone-swin-t)_(epoch-40) lb: 0.4576
                    '/output.csv',  # cascade_mask_rcnn(backbone-swin-s)_(epoch-36)_재경님꺼 lb:0.4151
#                   '/output12.csv', # detectors
#                   '/output123.csv', # cascade rcnn
                    '/submission_10.csv', # mask_rcnn_(backbone-swin)_(epoch-10)_(load_from_github_model) lb:0.4337
                    '/vfnet.csv', # vfnet 동주님꺼 lb: 0.4587
#                     '/vfnet_18.csv', # vfnet 동주님꺼 lb: 0.44
                    '/vfnet_24.csv', # vfnet 동주님꺼 lb: 0.4640
                    '/cascade_mask_rcnn_(backbone-swin)_(epoch-10)_(load_from_github_model)_(image_size=1333x800).csv', # lb:0.4640
#                     '/universenet101_base_20epoch.csv', # lb: 0.5168
#                     '/cascade_mask_rcnn_(backbone-swin)_(epoch-10)_(load_from_github_model)_(image_size=1024x1024_추가학습).csv'# 안내봄
    '/universenet101_pseudo (1).csv'
] 

submission_df = [pd.read_csv(dir+ file) for file in submission_files]

In [32]:
image_ids = submission_df[0]['image_id'].tolist()

In [33]:
annotation = '../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [34]:
prediction_strings = []
file_names = []
iou_thr = 0.6

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
#         boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=0.0001)
        labels = np.array(labels, dtype='int64')
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [35]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,8 0.6958047151565552 173.77989196777344 235.76...,batch_01_vt/0021.jpg
1,8 0.6081427335739136 158.21490478515625 61.387...,batch_01_vt/0028.jpg
2,1 0.5900605916976929 70.49398040771484 168.161...,batch_01_vt/0031.jpg
3,8 0.6474259495735168 42.533748626708984 131.17...,batch_01_vt/0032.jpg
4,8 0.9881086945533752 149.92315673828125 0.5436...,batch_01_vt/0070.jpg
